tensor是pytorch中用于对模型输入输出进行编码的一种数据类型

它和ndarray很相似，然而tensor可以使用GPU加速，还能执行自动微分

In [6]:
import torch
import numpy as np

创建一个tensor有很多种方法


In [2]:
# 直接通过数据创建
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

In [3]:
# 通过np数组创建
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [4]:
# 借助其他tensor创建，会保留tensor的属性但重写其数值
x_ones = torch.ones_like(x_data)
print(f"Ones Tensor:\n {x_ones}\n")
# 通过设置dtype来重写变量的类型
x_rand = torch.rand_like(x_data, dtype=torch.float)
print(f"Random Tensor:\n {x_rand}\n")


Ones Tensor:
 tensor([[1, 1],
        [1, 1]])

Random Tensor:
 tensor([[0.1669, 0.6030],
        [0.7654, 0.5285]])



In [5]:
# 通过随机数或者常量创建
# shape是一个元组，决定了tensor的形状
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)
print("Random Tensor:", rand_tensor)
print("Ones Tensor:", ones_tensor)
print("Zeros Tensor:", zeros_tensor)

Random Tensor: tensor([[0.9030, 0.3067, 0.8302],
        [0.9347, 0.3459, 0.5094]])
Ones Tensor: tensor([[1., 1., 1.],
        [1., 1., 1.]])
Zeros Tensor: tensor([[0., 0., 0.],
        [0., 0., 0.]])


tensor自带一些属性：大小、类型、设备，如下

In [7]:
tensor = torch.rand(3, 4)
print(f"Shape:{tensor.shape}\nDatatype:{tensor.dtype}\nDevice Stored{tensor.device} ")

Shape:torch.Size([3, 4])
Datatype:torch.float32
Device Storedcpu 


默认情况下tensor在CPU上创建，需要显式调用才能转到GPU上

In [9]:
torch.cuda.is_available()

True

In [8]:
# 如果有gpu，则把tensor移到GPU上
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

tensor的操作非常类似ndarray的操作，如下所示：

In [14]:
tensor = torch.rand(4, 4)
print(f"第一行: {tensor[0]}")
print(f"第一列: {tensor[:, 0]}")
# ...和:的作用几乎一致
print(f"最后一列: {tensor[..., -1]}")
tensor[:, 1] = 0
print(tensor)

第一行: tensor([0.4913, 0.7721, 0.1531, 0.3529])
第一列: tensor([0.4913, 0.3727, 0.6369, 0.4981])
最后一列: tensor([0.3529, 0.1438, 0.1039, 0.4589])
tensor([[0.4913, 0.0000, 0.1531, 0.3529],
        [0.3727, 0.0000, 0.0983, 0.1438],
        [0.6369, 0.0000, 0.8400, 0.1039],
        [0.4981, 0.0000, 0.7362, 0.4589]])


拼接tensor也是一件很容易的事情，还能指定拼接依据的维度（和pandas一致）

In [15]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[0.4913, 0.0000, 0.1531, 0.3529, 0.4913, 0.0000, 0.1531, 0.3529, 0.4913,
         0.0000, 0.1531, 0.3529],
        [0.3727, 0.0000, 0.0983, 0.1438, 0.3727, 0.0000, 0.0983, 0.1438, 0.3727,
         0.0000, 0.0983, 0.1438],
        [0.6369, 0.0000, 0.8400, 0.1039, 0.6369, 0.0000, 0.8400, 0.1039, 0.6369,
         0.0000, 0.8400, 0.1039],
        [0.4981, 0.0000, 0.7362, 0.4589, 0.4981, 0.0000, 0.7362, 0.4589, 0.4981,
         0.0000, 0.7362, 0.4589]])


张量之间可以进行一些矩阵运算（api偏底层，但设计轻巧，灵活度很高）

In [16]:
# 计算两个张量的矩阵乘积 张量*自身转置
# y1 y2 y3是相同的结果
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)
# y3 = torch.ones_like(y1)
# torch.matmul(tensor, tensor.T, out=y3)
y3 = torch.matmul(tensor, tensor.T)

In [17]:
# 计算两个张量的对应元素乘积
# z1 z2 z3 是相同的结果
z1 = tensor * tensor
z2 = tensor.mul(tensor)
z3 = torch.mul(tensor, tensor)

单变量tensor可以通过item()方法转为python基本类型

In [18]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

4.885901927947998 <class 'float'>


in-place操作符，是把操作符的运算结果存储覆盖至其调用对象的操作符，这类操作符一般会加上一个下划线后缀，如add_() copy_() t_()

虽然in-place操作符省内存，但计算的时候容易覆盖历史结果，因此还是少用为好

In [19]:
print(tensor)
tensor.add_(5)
print(tensor)

tensor([[0.4913, 0.0000, 0.1531, 0.3529],
        [0.3727, 0.0000, 0.0983, 0.1438],
        [0.6369, 0.0000, 0.8400, 0.1039],
        [0.4981, 0.0000, 0.7362, 0.4589]])
tensor([[5.4913, 5.0000, 5.1531, 5.3529],
        [5.3727, 5.0000, 5.0983, 5.1438],
        [5.6369, 5.0000, 5.8400, 5.1039],
        [5.4981, 5.0000, 5.7362, 5.4589]])


在CPU上存储的tensor和numpy数组实际上底层存储位置是相同的，因此改变一个会改变另一个

Tensor -> Numpy Array

In [20]:
t = torch.ones(5)
print("t:", t)
n = t.numpy()
print("n:", n)

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


In [21]:
# 浅复制
t.add_(1)
print(t)
print(n)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


Numpy Array -> Tensor

In [22]:
n = np.ones(5)
t = torch.from_numpy(n)

In [23]:
np.add(n, 1, out=n)
print(t)
print(n)

tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
[2. 2. 2. 2. 2.]
